In [4]:
import pandas as pd
import time
import os
import re
from google import genai
from dotenv import load_dotenv
from openai import OpenAI

In [2]:
load_dotenv()
client = genai.Client(api_key=os.getenv("api_key"))

In [8]:
def parse_resposta(resposta):
    try:
        partes = resposta.split(";")
        if len(partes) != 3:
            return "Erro", "Formato inválido", "0"
        return partes[0].strip(), partes[1].strip(), partes[2].strip()
    except Exception:
        return "Erro", "Falha no parse", "0"

In [9]:
tamanho_lote = 1000

df = pd.read_excel("source data/database cbo.xlsx", sheet_name=1)

lote_path = "activities batch.txt"
if os.path.exists(lote_path):
    with open(lote_path, "r") as f:
        lote_atual = int(f.read().strip())
else:
    lote_atual = 0

In [ ]:
while True:
    inicio = lote_atual * tamanho_lote
    fim = min(inicio + tamanho_lote, len(df))

    if inicio >= len(df):
        print("Todos os lotes foram processados.")
        break

    df_lote = df.iloc[inicio:fim]
    print(f"Processando lote {lote_atual + 1}, linhas de {inicio} a {fim - 1}")

    resultados = []
    tempo_inicio = time.time()

    try:
        for idx, row in df_lote.iterrows():
            print(f"Processando linha {idx + 1} de {len(df)}")

            group = row["task_group"]
            task = row["task"]
            tarefa_texto = f"{group}:{task}"

            prompt_usuario = f"""
Você é um especialista em organização do trabalho.

A seguir, há uma descrição de tarefa associada a uma ocupação.

Tarefa: "{tarefa_texto}"

Classifique a tarefa com base em UMA descrição EXATA contida na lista abaixo:

Entrada de Informação - Identificar e Avaliar Informações Relevantes para o Trabalho - Estimar Características Quantificáveis de Produtos, Eventos ou Informações;
Entrada de Informação - Identificar e Avaliar Informações Relevantes para o Trabalho - Identificar Objetos, Ações ou Eventos;
Entrada de Informação - Identificar e Avaliar Informações Relevantes para o Trabalho - Inspecionar Equipamentos, Estruturas ou Materiais;
Entrada de Informação - Buscar e Receber Informações Relacionadas ao Trabalho - Obter Informações;
Entrada de Informação - Buscar e Receber Informações Relacionadas ao Trabalho - Monitorar Processos, Materiais ou Ambiente;

Interação com Outros - Administrar - Monitorar e Controlar Recursos;
Interação com Outros - Administrar - Realizar Atividades Administrativas;
Interação com Outros - Administrar - Recrutamento e Seleção de Funcionários;
Interação com Outros - Comunicar e Interagir - Auxiliar e Cuidar de Outras Pessoas;
Interação com Outros - Comunicar e Interagir - Comunicar com Pessoas Fora da Organização;
Interação com Outros - Comunicar e Interagir - Comunicar com Supervisores, Colegas ou Subordinados;
Interação com Outros - Comunicar e Interagir - Estabelecer e Manter Relacionamentos Interpessoais;
Interação com Outros - Comunicar e Interagir - Interpretar o Significado da Informação para Outros;
Interação com Outros - Comunicar e Interagir - Atuar para ou Trabalhar Diretamente com o Público;
Interação com Outros - Comunicar e Interagir - Resolver Conflitos e Negociar com Outros;
Interação com Outros - Comunicar e Interagir - Vender ou Influenciar Outros;

Coordenar, Desenvolver, Gerenciar e Aconselhar - Treinar e Ensinar Outros;
Coordenar, Desenvolver, Gerenciar e Aconselhar - Coordenar o Trabalho e as Atividades de Outros;
Coordenar, Desenvolver, Gerenciar e Aconselhar - Desenvolver e Construir Equipes;
Coordenar, Desenvolver, Gerenciar e Aconselhar - Orientar, Dirigir e Motivar Subordinados;
Coordenar, Desenvolver, Gerenciar e Aconselhar - Prestar Consultoria e Aconselhamento a Outros;

Processos Mentais - Processamento de Informação e Dados - Analisar Dados ou Informações;
Processos Mentais - Processamento de Informação e Dados - Avaliar Informações para Determinar Conformidade com Padrões;
Processos Mentais - Processamento de Informação e Dados - Julgar a Qualidade de Objetos, Serviços ou Pessoas;
Processos Mentais - Processamento de Informação e Dados - Processar Informações;
Processos Mentais - Raciocínio e Tomada de Decisão - Desenvolver Objetivos e Estratégias;
Processos Mentais - Raciocínio e Tomada de Decisão - Tomar Decisões e Resolver Problemas;
Processos Mentais - Raciocínio e Tomada de Decisão - Organizar, Planejar e Priorizar o Trabalho;
Processos Mentais - Raciocínio e Tomada de Decisão - Agendar Trabalhos e Atividades;
Processos Mentais - Raciocínio e Tomada de Decisão - Pensar Criativamente;
Processos Mentais - Raciocínio e Tomada de Decisão - Atualizar e Utilizar Conhecimentos Relevantes;

Resultado do Trabalho - Realização de Atividades Complexas e Técnicas - Documentar/Registrar Informações;
Resultado do Trabalho - Realização de Atividades Complexas e Técnicas - Projetar e Especificar Dispositivos Técnicos, Peças e Equipamentos;
Resultado do Trabalho - Realização de Atividades Complexas e Técnicas - Reparar e Manter Equipamentos Eletrônicos;
Resultado do Trabalho - Realização de Atividades Complexas e Técnicas - Reparar e Manter Equipamentos Mecânicos;
Resultado do Trabalho - Realização de Atividades Complexas e Técnicas - Trabalhar com Computadores;
Resultado do Trabalho - Realização de Atividades Físicas e Manuais - Controlar Máquinas e Processos;
Resultado do Trabalho - Realização de Atividades Físicas e Manuais - Manipular e Mover Objetos;
Resultado do Trabalho - Realização de Atividades Físicas e Manuais - Operar Veículos, Equipamentos ou Dispositivos Mecanizados;
Resultado do Trabalho - Realização de Atividades Físicas e Manuais - Realizar Atividades Físicas Gerais;

Seja objetivo, direto e sucinto.

Forneça uma justificativa breve que explique a classificação atribuída.

Forneça um grau de confiança para sua resposta que vai de 0 a 100 (somente o número).

Sua resposta deve seguir exatamente este formato, separados por ponto e vírgula: classificação exata;justificativa breve;grau de confiança (de 0 a 100).
"""

            try:
                response = client.models.generate_content(
                    model="gemini-2.0-flash",
                    contents=prompt_usuario,
                )
                resposta = response.text
            except Exception as e:
                print(f"Erro ao obter resposta do modelo: {e}")
                resposta = f"Erro: {e}"

            try:
                classificacao, justificativa, confianca = parse_resposta(resposta)
            except Exception as e:
                print(f"Erro ao processar resposta: '{resposta}'. Erro: {e}")
                classificacao, justificativa, confianca = "Erro", resposta, "0"

            resultados.append({
                "Tarefa": tarefa_texto,
                "Classificação": classificacao,
                "justificativa": justificativa,
                "confianca": confianca
            })

            time.sleep(0)

        df_resultado = pd.DataFrame(resultados)
        nome_arquivo = f"activities batches/batch{lote_atual + 1}.xlsx"
        tempo_fim = time.time()
        duracao = tempo_fim - tempo_inicio

        df_stats = pd.DataFrame([{
            "Lote": lote_atual + 1,
            "Inicio": inicio,
            "Fim": fim - 1,
            "Linhas processadas": len(df_lote),
            "Tempo (s)": round(duracao, 2)
        }])

        with pd.ExcelWriter(nome_arquivo, engine="xlsxwriter") as writer:
            df_resultado.to_excel(writer, index=False, sheet_name="Resultados")
            df_stats.to_excel(writer, index=False, sheet_name="Stats")

        with open(lote_path, "w") as f:
            f.write(str(lote_atual + 1))

        print(f"\n✅ Lote {lote_atual + 1} finalizado e salvo como {nome_arquivo}. Tempo de execução: {round(duracao, 2)} segundos.\n")
        lote_atual += 1

    except Exception as e:
        print(f"Erro ao processar o lote {lote_atual + 1}: {e}")
        break

Processando lote 1, linhas de 0 a 999
Processando linha 1 de 53409
Processando linha 2 de 53409
Processando linha 3 de 53409
Processando linha 4 de 53409
Processando linha 5 de 53409
Processando linha 6 de 53409
Processando linha 7 de 53409
Processando linha 8 de 53409
Processando linha 9 de 53409
Processando linha 10 de 53409
Processando linha 11 de 53409
Processando linha 12 de 53409
Processando linha 13 de 53409
Processando linha 14 de 53409
Processando linha 15 de 53409
Processando linha 16 de 53409
Processando linha 17 de 53409
Processando linha 18 de 53409
Processando linha 19 de 53409
Processando linha 20 de 53409
Processando linha 21 de 53409
Processando linha 22 de 53409
Processando linha 23 de 53409
Processando linha 24 de 53409
Processando linha 25 de 53409
Processando linha 26 de 53409
Processando linha 27 de 53409
Processando linha 28 de 53409
Processando linha 29 de 53409
Processando linha 30 de 53409
Processando linha 31 de 53409
Processando linha 32 de 53409
Processando

In [3]:
pasta = "activities batches"

arquivos_xlsx = [
    os.path.join(pasta, f)
    for f in os.listdir(pasta)
    if f.endswith(".xlsx")
]

dfs = []

for arquivo in arquivos_xlsx:
    print(f"Lendo: {arquivo}")
    df = pd.read_excel(arquivo)
    df["arquivo_origem"] = os.path.basename(arquivo) 
    dfs.append(df)

df_todos = pd.concat(dfs, ignore_index=True)

arquivo_final = os.path.join("results", "all_activities.xlsx")

with pd.ExcelWriter(arquivo_final, engine="xlsxwriter") as writer:
    df_todos.to_excel(writer, index=False, sheet_name="Results")

print(f"Arquivo final salvo em: {arquivo_final}") 

Lendo: activities batches\batch1.xlsx
Lendo: activities batches\batch10.xlsx
Lendo: activities batches\batch11.xlsx
Lendo: activities batches\batch12.xlsx
Lendo: activities batches\batch13.xlsx
Lendo: activities batches\batch14.xlsx
Lendo: activities batches\batch15.xlsx
Lendo: activities batches\batch16.xlsx
Lendo: activities batches\batch17.xlsx
Lendo: activities batches\batch18.xlsx
Lendo: activities batches\batch19.xlsx
Lendo: activities batches\batch2.xlsx
Lendo: activities batches\batch20.xlsx
Lendo: activities batches\batch21.xlsx
Lendo: activities batches\batch22.xlsx
Lendo: activities batches\batch23.xlsx
Lendo: activities batches\batch24.xlsx
Lendo: activities batches\batch25.xlsx
Lendo: activities batches\batch26.xlsx
Lendo: activities batches\batch27.xlsx
Lendo: activities batches\batch28.xlsx
Lendo: activities batches\batch29.xlsx
Lendo: activities batches\batch3.xlsx
Lendo: activities batches\batch30.xlsx
Lendo: activities batches\batch31.xlsx
Lendo: activities batches\ba

In [5]:
def sample_size(populacao, margem_erro=0.05, nivel_confianca=0.95, proporcao=0.5):

    z = 1.96
    
    numerador = populacao * (z ** 2) * proporcao * (1 - proporcao)
    denominador = (populacao - 1) * (margem_erro ** 2) + (z ** 2) * proporcao * (1 - proporcao)
    
    tamanho_amostra = numerador / denominador
    return math.ceil(tamanho_amostra) 

populacao = 53409

tamanho_amostra = sample_size(populacao)

In [10]:
sample = df_todos.sample(n=tamanho_amostra)

sample.to_excel('sample/sample_activities.xlsx', index=False)

print(f"Amostra de {tamanho_amostra} registros salva em 'sample/sample_activities.xlsx'.")

Amostra de 382 registros salva em 'sample/sample_activities.xlsx'.
